In [1]:
import pandas as pd
import numpy as np
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt
import re
import docx
import docx2txt
from fnmatch import fnmatch
from selenium.webdriver.support.ui import WebDriverWait
# Importar a classe que contém as funções e aplicar um alias
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, WebDriverException, ElementClickInterceptedException, InvalidElementStateException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import datetime
from selenium import webdriver
import time
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Alexandre Cunha\AppData\Local\Temp\ipykernel_7116\543018007.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [60]:
def abre_lattes():
    #selecionada nova aba
    window_after = driver.window_handles[0]
    #ativa nova aba
    driver.switch_to.window(window_after)
    #Abre a pagina do Lattes
    driver.get ('http://buscatextual.cnpq.br/buscatextual/busca.do')
    return()

def nome_busca_lattes(nome):
    #Marca caixa de seleção de demais informações
    driver.find_element(By.ID, "buscarDemais").click()
    #limpa caixa de seleção
    driver.find_element(By.ID, "textoBusca").clear()
    #Escreve na caixa de seleção 
    driver.find_element(By.ID, "textoBusca").send_keys(nome)
    #Clica enter
    driver.find_element(By.ID, "textoBusca").send_keys(Keys.ENTER)
    #tempo para busca
    time.sleep(2)
    #conta quantos itens achados
    conta=len(driver.find_elements(By.TAG_NAME, 'li'))
    if conta==1:
        print(driver.find_elements(By.TAG_NAME, 'li')[0].text)
        print(nome)
        resposta=input('O nome esta correto? ')
        driver.find_elements(By.TAG_NAME, 'li')[0].find_element(By.TAG_NAME, 'a').click()
    else:
        cont=0
        for nomes in driver.find_elements(By.TAG_NAME, 'li')[:5]:
            print(nomes.text)
            print(cont)
            cont+=1
        resposta=input('Deseja pesquisar algum nome? ')
        if resposta=='sim':
            num=int(input('Digite o numero da busca:'))
            driver.find_elements(By.TAG_NAME, 'li')[num].find_element(By.TAG_NAME, 'a').click()
        else:
            abre_lattes()
    if 'Nenhum resultado' in driver.find_elements(By.CLASS_NAME, 'tit_form').text:
        print(nome)
        driver.find_element(By.ID, 'botaoBuscaFiltros').click()
        resposta2=input('Nenhum resultado encontrado deseja pesquisar no escavador?')
    return(resposta)

def entra_lattes():
    #abre aba do lattes do usuario
    driver.find_element(By.ID, 'idbtnabrircurriculo').click()
    time.sleep(2)
    #aba corrente
    default_handle = driver.current_window_handle
    #selecionada nova aba
    window_after = driver.window_handles[1]
    #ativa nova aba
    driver.switch_to.window(window_after)
    return()

def nome_busca_lattes2(nome):
    df_todos1=pd.read_pickle('df_pesquisadores.pickle')
    #pesquisa no lattes:
    abre_lattes()
    #Marca caixa de seleção de demais informações
    driver.find_element(By.ID, 'buscarDemais').click()
    #limpa caixa de seleção
    driver.find_element(By.ID, 'textoBusca').clear()
    #Escreve na caixa de seleção 
    driver.find_element(By.ID, 'textoBusca').send_keys(nome)
    #Clica enter
    driver.find_element(By.ID, 'textoBusca').send_keys(Keys.ENTER)
    #tempo para busca
    time.sleep(2)
    #conta quantos itens achados
    conta=len(driver.find_elements(By.TAG_NAME, 'li'))
    if conta==1:
        print(driver.find_elements(By.TAG_NAME, 'li')[0].text)
        print(nome)
        driver.find_elements(By.TAG_NAME, 'li')[0].find_element(By.TAG_NAME, 'a').click()
        #continua lattes
        entra_lattes()
        df_uni=df_padrao_unitario(nome)
        resposta='sim'
        if df_todos1[df_todos1['Link_lattes']==df_uni['Link_lattes'][0]]['Link_lattes'].count()>0:
            #Nova Busca
            limpa_page()
            pass
        else:
            df_todos1=pd.concat([df_todos1,df_uni])
            df_todos1.to_pickle('df_pesquisadores.pickle')
            #Nova Busca
            limpa_page()
    else:
        cont=0
        for nomes in driver.find_elements(By.TAG_NAME, 'li')[:5]:
            pass
        print(nome+' não foi')
        resposta='não'
        pass
    if 'Nenhum resultado' in driver.find_element(By.CLASS_NAME, 'tit_form').text:
        print(nome)
        driver.find_element(By.ID, 'botaoBuscaFiltros').click()
        print(nome+'não foi')
        resposta='não'
        pass
    return(resposta)

def df_padrao_unitario(nome_bolsista):
    lista_resumo=[]
    lista_ultima_atualização=[]
    lista_acadêmica=[]
    lista_Complementar=[]
    lista_Profissional=[]
    lista_pesquisa=[]
    lista_area_atuacao=[]
    lista_Idiomas=[]
    lista_Produções=[]
    lista_link=[]
    #imprime resumo
    time.sleep(2)
    lista_resumo.append(driver.find_element(By.CLASS_NAME, 'resumo').text)
    #Tras o ano do PIBIC
    #print('Ano da Bolsa:',list(df_bics_final1[df_bics_final1['Nome_corretos']==nome]['Ano'])[0])
    #Verificar ultima atualização
    for ultima in driver.find_element(By.CLASS_NAME, 'informacoes-autor').find_elements(By.TAG_NAME, 'li'):
        if 'Última' in ultima.text:
            lista_ultima_atualização.append(ultima.text)
            print('\n')
            print(ultima.text)
    resumo=0
    ultima_atualização=0
    acadêmica=0
    Complementar=0
    Profissional=0
    pesquisa=0
    area_atuacao=0
    Idiomas=0
    Produções=0
    link_lattes=0
    #Verifica titulos
    for titulo in driver.find_elements(By.CLASS_NAME, 'title-wrapper'):
        if 'Formação acadêmica' in titulo.text:
            lista_acadêmica.append(titulo.text.split('\n'))
            acadêmica=1
        if 'Formação Complementar' in titulo.text:
            if ('Formação Complementar'in titulo.text.split('\n')[0]) and len(titulo.text.split('\n')[0])<=25:
                lista_Complementar.append(titulo.text.split('\n'))
                Complementar=1
        if 'Atuação Profissional' in titulo.text:
            lista_Profissional.append(titulo.text.split('\n'))
            Profissional=1
        if 'Projetos de pesquisa' in titulo.text.split('\n')[0]:
            lista_pesquisa.append(titulo.text.split('\n'))
            pesquisa=1
        if 'Áreas de atuação' in titulo.text:
            if 'Áreas de atuação'in titulo.text.split('\n')[0]:
                lista_area_atuacao.append(titulo.text.split('\n'))
                area_atuacao=1
        if 'Idiomas'in titulo.text:
            if 'Idiomas'in titulo.text.split('\n')[0]:
                lista_Idiomas.append(titulo.text.split('\n'))
                Idiomas=1
        if 'Produções' in titulo.text:
            if 'Produções'in titulo.text.split('\n')[0]:
                lista_Produções.append(titulo.text.split('\n'))
                Produções=1
        if driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[1]/ul/li[1]').text!='' and link_lattes==0:
            lista_link.append(driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[1]/ul/li[1]').text)
            link_lattes=1
    if acadêmica==0:
        lista_acadêmica.append('')
    if Complementar==0:
        lista_Complementar.append('')
    if Profissional==0:
        lista_Profissional.append('')
    if pesquisa==0:
        lista_pesquisa.append('')
    if area_atuacao==0:
        lista_area_atuacao.append('')
    if Idiomas==0:
        lista_Idiomas.append('')
    if Produções==0:
        lista_Produções.append('')
    if link_lattes==0:
        lista_link.append('')
    df_padrao=pd.DataFrame([])
    print(nome_bolsista)
    df_padrao['Resumo']=lista_resumo
    df_padrao['Ultima_atualização']=lista_ultima_atualização
    df_padrao['Academico']=lista_acadêmica
    df_padrao['Profissional']=lista_Profissional
    df_padrao['Pesquisa']=lista_pesquisa
    df_padrao['Produção']=lista_Produções
    df_padrao['Idiomas']=lista_Idiomas
    df_padrao['Complementar']=lista_Complementar
    df_padrao['Area_Atuação']=lista_area_atuacao
    df_padrao['Link_lattes']=lista_link
    df_padrao['Bolsista']=nome_bolsista
    return(df_padrao)

def limpa_page():
    window_after = driver.window_handles[1]
    #ativa nova aba
    driver.switch_to.window(window_after)
    driver.close()
    window_after = driver.window_handles[0]
    #ativa nova aba
    driver.switch_to.window(window_after)
    #botão fechar
    driver.find_element(By.ID, 'idbtnfechar').click()
    #nova consulta
    driver.find_element(By.ID, 'botaoBuscaFiltros').click()
    return()

In [25]:
df_lista_teste=pd.read_excel('Teste_scraping_lattes.xlsx')
df_lista_teste

,IDENTIFICAÇÃO FUNCIONAL,NOME COMPLETO
0,672008,ANA MARIA BOTELHO MARINHO DA CUNHA
1,1142303,ANDRÉA CAMARDELLA DE LIMA RIZZO
2,1318161,ARNALDO ALCOVER NETO
3,1566884,CLAUDIA DUARTE DA CUNHA
4,1354812,CLAUDIO LUIZ SCHNEIDER
5,2047734,ELLEN CRISTINE GIESE
6,1691834,ELVES MATIOLO
7,2127185,FABIO GUISTI AZEVEDO DE BRITTO
8,672016,FERNANDO ANTONIO FREITAS LINS
9,671079,FRANCISCO MARIANO DA ROCHA DE SOUZA LIMA


In [ ]:
lista_unicos=list(df_lista_teste['NOME COMPLETO'])
lista_unicos

In [27]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Alexandre Cunha\AppData\Local\Temp\ipykernel_9472\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [61]:
for nome in lista_unicos[:10]:
    nome_busca_lattes2(nome)

Ana Maria Botelho Marinho da Cunha 
Especialização em Engenharia de Produção pelo Programa de Engenharia de Produção, Brasil(1979)
Tecnologista Senior III do Centro de Tecnologia Mineral , Brasil
ANA MARIA BOTELHO MARINHO DA CUNHA


Última atualização do currículo em 17/12/2019
ANA MARIA BOTELHO MARINHO DA CUNHA
Andréa Camardella de Lima Rizzo 
Doutorado em Engenharia Química pela Universidade Federal do Rio de Janeiro, Brasil(2008)
Tecnologista Senior do Centro de Tecnologia Mineral , Brasil
ANDRÉA CAMARDELLA DE LIMA RIZZO


Última atualização do currículo em 06/12/2021
ANDRÉA CAMARDELLA DE LIMA RIZZO
Arnaldo Alcover Neto 
Doutorado em Geociências (Geoquímica e Geotectônica) pela Universidade de São Paulo, Brasil(1997)
Pesquisador Titular do Centro de Tecnologia Mineral , Brasil
ARNALDO ALCOVER NETO


Última atualização do currículo em 07/06/2022
ARNALDO ALCOVER NETO
Cláudia Duarte da Cunha 
Doutorado em Tecnologia de Processos Químicos e Bioquímicos pela UFRJ - Escola de Química, Bra

In [2]:
df_pesquisadores=pd.read_pickle('df_pesquisadores.pickle')
df_pesquisadores

,Resumo,Ultima_atualização,Academico,Profissional,Pesquisa,Produção,Idiomas,Complementar,Area_Atuação,Link_lattes,Bolsista
0,Atualmente é tecnologista senior iii do Centro...,Última atualização do currículo em 17/12/2019,"[Formação acadêmica/titulação, 1976 - 1979, Es...","[Atuação Profissional, , Agência Nacional do P...",,"[Produções, , Produção bibliográfica, Artigos ...","[Idiomas, Inglês, Compreende Razoavelmente, Fa...",,"[Áreas de atuação, 1., Grande área: Engenharia...",Endereço para acessar este CV: http://lattes.c...,ANA MARIA BOTELHO MARINHO DA CUNHA
0,Possui graduação em Engenharia Química pela Un...,Última atualização do currículo em 06/12/2021,"[Formação acadêmica/titulação, 2004 - 2008, Do...","[Atuação Profissional, , Centro de Tecnologia ...","[Projetos de pesquisa, 2018 - Atual, Evolução ...","[Produções, , Produção bibliográfica, Artigos ...","[Idiomas, Português, Compreende Bem, Fala Bem,...","[Formação Complementar, 2021 - 2021, Pensament...","[Áreas de atuação, 1., Grande área: Engenharia...",Endereço para acessar este CV: http://lattes.c...,ANDRÉA CAMARDELLA DE LIMA RIZZO
0,Possui graduação em Bacharelado Em Química pel...,Última atualização do currículo em 07/06/2022,"[Formação acadêmica/titulação, 1991 - 1996, Do...","[Atuação Profissional, , Centro de Tecnologia ...","[Projetos de pesquisa, 2019 - Atual, DESENVOLV...","[Produções, , Produção bibliográfica, Artigos ...","[Idiomas, Inglês, Compreende Razoavelmente, Fa...",[Formação Complementar],"[Áreas de atuação, 1., Grande área: Ciências E...",Endereço para acessar este CV: http://lattes.c...,ARNALDO ALCOVER NETO
0,"Possui graduação em Engenharia Quimica, mestra...",Última atualização do currículo em 28/07/2022,"[Formação acadêmica/titulação, 1999 - 2004, Do...","[Atuação Profissional, , Centro de Tecnologia ...","[Projetos de pesquisa, 2022 - Atual, Aproveita...","[Produções, , Produção bibliográfica, Artigos ...","[Idiomas, Português, Compreende Bem, Fala Bem,...","[Formação Complementar, 2016 - 2016, Noções bá...","[Áreas de atuação, 1., Grande área: Engenharia...",Endereço para acessar este CV: http://lattes.c...,CLAUDIA DUARTE DA CUNHA
0,Atua como Pesquisadora no Centro de Tecnologia...,Última atualização do currículo em 30/07/2022,"[Formação acadêmica/titulação, 2005 - 2008, Do...","[Atuação Profissional, , Serviço de Apoio às M...","[Projetos de pesquisa, 2021 - Atual, REDUCTIVE...","[Produções, , Produção bibliográfica, Citações...","[Idiomas, Inglês, Compreende Bem, Fala Bem, Lê...","[Formação Complementar, 2022 - 2022, Bioleachi...","[Áreas de atuação, 1., Grande área: Ciências E...",Endereço para acessar este CV: http://lattes.c...,ELLEN CRISTINE GIESE
0,Possui graduação em Engenharia de Minas pela U...,Última atualização do currículo em 01/12/2022,"[Formação acadêmica/titulação, 2005 - 2008, Do...","[Atuação Profissional, , Centro de Tecnologia ...","[Projetos de pesquisa, 2013 - Atual, PROTERRAR...","[Produções, , Produção bibliográfica, Artigos ...","[Idiomas, Inglês, Compreende Bem, Fala Bem, Lê...","[Formação Complementar, 2013 - 2013, Gestão de...","[Áreas de atuação, 1., Grande área: Engenharia...",Endereço para acessar este CV: http://lattes.c...,ELVES MATIOLO
0,Graduado em Engenharia Metalúrgica pela PUC-Ri...,Última atualização do currículo em 03/07/2020,"[Formação acadêmica/titulação, 1990 - 1995, Do...","[Atuação Profissional, , Companhia de Pesquisa...",,"[Produções, , Produção bibliográfica, Citações...","[Idiomas, Francês, Compreende Razoavelmente, F...","[Formação Complementar, 2001 - 2001, Prêmio Na...","[Áreas de atuação, 1., Grande área: Engenharia...",Endereço para acessar este CV: http://lattes.c...,FERNANDO ANTONIO FREITAS LINS
0,Possui graduação em Engenharia Metalúrgica e I...,Última atualização do currículo em 03/06/2016,"[Formação acadêmica/titulação, 2009 - 2013, Do...","[Atuação Profissional, , Centro de Tecnologia ...","[Projetos de pesquisa, 2015 - Atual, AVALIAÇÃO...","[Produções, , Produção bibliográfica, Ca

In [4]:
df_pesquisadores.to_excel('df_pesquisadores.xlsx')